In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplcursors # Use this is for creating a cursor-interactive plot with "%matplotlib notebook"
from sklearn.decomposition import NMF # Use this for training Non-negative Matrix Factorization
from sklearn.utils.extmath import randomized_svd # Use this for training Singular Value Decomposition
from sklearn.manifold import TSNE # Use this for training t-sne manifolding

In [2]:
dir = './최종데이터/'
#df_product = pd.read_csv(dir + 'basic_data_edit.csv', usecols=['00.상품_URL','00.상품코드','01.브랜드','02.상품명','03.가격','04.제품 주요 사양','05.모든 성분','06.총 평점','07.리뷰 개수','08_1.별점 1점','08_2.별점 2점','08_3.별점 3점','08_4.별점 4점','08_5.별점 5점','09_1.피부타입_건성','09_2.피부타입_복합성','09_3.피부타입_지성','10_1.피부고민_보습','10_2.피부고민_진정','10_3.피부고민_주름/미백','11_1.피부자극_없음','11_2.피부자극_보통','11_3.피부자극_있음'],encoding='cp949')
df_product = pd.read_csv(dir + 'basic_data_img.csv', usecols=['00.상품코드','00.상품_URL','00.이미지_URL','01.브랜드','02.상품명','03.가격','04.제품 주요 사양','05.모든 성분','06.총 평점','07.리뷰 개수','08_1.별점 1점','08_2.별점 2점','08_3.별점 3점','08_4.별점 4점','08_5.별점 5점','09_1.피부타입_건성','09_2.피부타입_복합성','09_3.피부타입_지성','10_1.피부고민_보습','10_2.피부고민_진정','10_3.피부고민_주름/미백','11_1.피부자극_없음','11_2.피부자극_보통','11_3.피부자극_있음'],encoding='cp949')
df_review = pd.read_csv(dir + 'review_data_edit.csv', usecols=['00.상품코드','01.리뷰어 닉네임','02_1.피부 타입','02_2.피부 톤','02_3.피부 고민','03.리뷰 평점','04.피부타입 | 피부고민 | 자극도','05.리뷰'],encoding='cp949')
print('done')

done


In [3]:
product_list = df_review['00.상품코드']
product_list = product_list.unique()
len(product_list)

430

In [7]:
#column 이름 변경 및 결측치 제거
df_review.columns = ['code','user','type','tone','problem','rating','feature','review']
df_review = df_review.dropna(axis=0)

In [8]:
#column 이름 변경 및 결측치 제거
df_product.columns = ['code','00.상품_URL','00.이미지_URL','01.브랜드','02.상품명','03.가격','04.제품 주요 사양','05.모든 성분','06.총 평점','07.리뷰 개수','08_1.별점 1점','08_2.별점 2점','08_3.별점 3점','08_4.별점 4점','08_5.별점 5점','09_1.피부타입_건성','09_2.피부타입_복합성','09_3.피부타입_지성','10_1.피부고민_보습','10_2.피부고민_진정','10_3.피부고민_주름/미백','11_1.피부자극_없음','11_2.피부자극_보통','11_3.피부자극_있음']
df_product = df_product.dropna(axis=0)
df_product = df_product.drop_duplicates(['code'])

In [9]:
len(df_product)

518

In [10]:
df_review

,code,user,type,tone,problem,rating,feature,review
0,A000000147339,요다,건성,웜톤,민감성 | 아토피 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",개인적으로 성분으로 마케팅하는 제품들을 그렇게 좋아하진 않습니다. 특히 토너는 순한...
1,A000000147339,올리브양,복합성,쿨톤,모공 | 민감성 | 주름 | 탄력 | 트러블 | 홍조,5.0,"('건성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",복합성 피부에 많이 민감한 피부입니다. 요즘 봄 바람에 아침 저녁으로 기온차가 심해...
2,A000000147339,뚜비이,건성,쿨톤,민감성 | 잡티 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",지난번 올영체험단으로 제공받아 사용해보고 너무 좋아서 다시 구매했습니당!\r\n저는...
3,A000000147339,시래우,복합성,겨울쿨톤,다크서클 | 모공 | 미백 | 민감성 | 블랙헤드 | 아토피 | 잡티 | 주름 | ...,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",날씨 더워지면 광대가 뜨거워지면서 내 얼굴이 호빵맨이 되어가는 것 같고 좀 더 더워...
5,A000000147339,효지니,복합성,쿨톤,민감성 | 블랙헤드 | 아토피 | 잡티 | 트러블,5.0,"('복합성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",요즘에 피부가 너무 나빠져서 잘 맞는 화장품 찾아볼려고 이것저것 구매를 해봤어요! ...
...,...,...,...,...,...,...,...,...
158580,A000000148095,겨울별,지성,쿨톤,각질 | 미백 | 블랙헤드 | 잡티 | 주름 | 트러블,5.0,"('지성에 좋아요 | 진정에 좋아요 | 자극없이 순해요',)",봄철에딱인 속건조토너 흑미탄탄토너를 일주일정도 써보았습니다. 뷰티유튜버가 왜추천을했...
158582,A000000148095,moon219,복합성,쿨톤,모공 | 트러블,5.0,"('건성에 좋아요 | 보습에 좋아요 | 자극없이 순해요',)",제품이 순하기도 하고 성분이 좋아서 그런지 토너팩을 매일같이 하니까 정말 많이 진정...
158583,A000000148095,리파리해파리파리,건성,웜톤,각질 | 모공 | 미백 | 블랙헤드 | 잡티 | 주름 | 탄력 | 트러블,4.0,"('복합성에 좋아요 | 주름/미백에 좋아요 | 자극없이 순해요',)",저는 올리브영체험단을 몇번하면서\r\n항상 제품과 리뷰작성바란다는 안내문만 받아보다...
158586,A000000148095,luv9,건성,웜톤,각질 | 민감성 | 주름 | 홍조,4.0,"('건성에 좋아요 | 보습에 좋아요 | 자극없이 순해요',)",체험단 신청해서 받아보게 되었는데! 직접 손편지로 리뷰작성안내 해주신 브랜드는 처음...


In [11]:
user_list = df_review['code']
user_list = user_list.unique()
user_list[0]
df_review_user = df_review[df_review['code']==user_list[0]]

problem_dict = {}
for problem in df_review_user['problem']:
    problem_list = problem.split('|')
    for i in problem_list:
        i = i.replace(" ","")
        if i in problem_dict:
            problem_dict[i] += 1
        else:
            problem_dict[i] = 1
problem_dict_sort = sorted(problem_dict.items(),key = lambda x: x[1],reverse = True)
problem_dict_sort[0][0]


'트러블'

In [13]:
user_list = df_review['code']
user_list = user_list.unique()
user_list[0]
df_review_user = df_review[df_review['code']==user_list[0]]

#최다빈출 type
type_freq = df_review_user['type'].value_counts()
#최다빈출 tone
tone_freq = df_review_user['tone'].value_counts()
#최다빈출 problem
problem_dict = {}
for problem in df_review_user['problem']:
    problem_list = problem.split('|')
    for i in problem_list:
        i = i.replace(" ","")
        if i in problem_dict:
            problem_dict[i] += 1
        else:
            problem_dict[i] = 1
problem_dict_sort = sorted(problem_dict.items(),key = lambda x: x[1],reverse = True)



type = type_freq.index[0]
tone = tone_freq.index[0]
problem = ''
for i, problems in enumerate(problem_dict_sort):
    problem += problems[0] + ' '
    if i==20:
        break
        
        
print(type)
print(tone)
print(type+' '+tone+' '+problem)


복합성
웜톤
복합성 웜톤 트러블 모공 민감성 잡티 블랙헤드 미백 홍조 탄력 각질 피지과다 주름 다크서클 아토피 


In [15]:
df_feature = pd.DataFrame(columns = ['code','feature'])


product_list = df_review['code']
product_list = product_list.unique()
index = 0
for product in product_list:
    df_review_user = df_review[df_review['code']==product]
    
    #최다빈출 type
    type_freq = df_review_user['type'].value_counts()
    #최다빈출 tone
    tone_freq = df_review_user['tone'].value_counts()
    #최다빈출 problem
    problem_dict = {}
    for problem in df_review_user['problem']:
        problem_list = problem.split('|')
        for i in problem_list:
            i = i.replace(" ","")
            if i in problem_dict:
                problem_dict[i] += 1
            else:
                problem_dict[i] = 1
    problem_dict_sort = sorted(problem_dict.items(),key = lambda x: x[1],reverse = True)



    type = type_freq.index[0]
    tone = tone_freq.index[0]
    problem = ''
    for i, problems in enumerate(problem_dict_sort):
        problem += problems[0] + ' '
        if i==4:
            break

    feature = type+' '+tone+' '+problem
    
    print(index)
    print(product)
    print(feature)
    
    df_feature.loc[str(index)] = [product,feature]
    
    index += 1

0
A000000147339
복합성 웜톤 트러블 모공 민감성 잡티 블랙헤드 
1
A000000137180
복합성 웜톤 트러블 모공 블랙헤드 잡티 민감성 
2
A000000141704
복합성 웜톤 트러블 모공 잡티 블랙헤드 민감성 
3
A000000104007
복합성 웜톤 모공 트러블 잡티 민감성 블랙헤드 
4
A000000146589
복합성 웜톤 트러블 모공 민감성 잡티 블랙헤드 
5
A000000016686
복합성 웜톤 모공 잡티 트러블 민감성 블랙헤드 
6
A000000161508
복합성 웜톤 모공 트러블 블랙헤드 잡티 각질 
7
A000000145578
복합성 웜톤 모공 트러블 잡티 민감성 블랙헤드 
8
A000000126862
복합성 쿨톤 트러블 모공 블랙헤드 민감성 잡티 
9
A000000156512
복합성 웜톤 잡티 트러블 모공 블랙헤드 민감성 
10
A000000150946
복합성 웜톤 트러블 모공 잡티 민감성 블랙헤드 
11
A000000148899
복합성 쿨톤 민감성 트러블 모공 잡티 주름 
12
A000000129008
복합성 웜톤 민감성 모공 트러블 블랙헤드 잡티 
13
A000000147524
복합성 웜톤 트러블 모공 블랙헤드 잡티 민감성 
14
A000000154177
복합성 웜톤 트러블 모공 블랙헤드 각질 잡티 
15
A000000157294
복합성 웜톤 모공 잡티 블랙헤드 민감성 주름 
16
A000000143827
복합성 웜톤 모공 트러블 잡티 각질 민감성 
17
A000000117541
복합성 쿨톤 모공 잡티 트러블 민감성 블랙헤드 
18
A000000012809
복합성 웜톤 민감성 잡티 트러블 모공 홍조 
19
A000000147387
복합성 웜톤 트러블 모공 민감성 잡티 블랙헤드 
20
A000000010474
복합성 웜톤 모공 민감성 트러블 블랙헤드 잡티 
21
A000000147313
복합성 웜톤 블랙헤드 모공 잡티 트러블 미백 
22
A000000106421
복합성 웜톤 모공 블랙헤드 잡티 트러블 민감성 
23
A0000

258
A000000129303
복합성 웜톤 모공 트러블 홍조 미백 민감성 
259
A000000129035
복합성 웜톤 트러블 모공 잡티 민감성 미백 
260
A000000136077
복합성 웜톤 모공 트러블 잡티 블랙헤드 민감성 
261
A000000135772
복합성 웜톤 블랙헤드 잡티 모공 미백 주름 
262
A000000128966
복합성 웜톤 모공 트러블 민감성 블랙헤드 잡티 
263
A000000137115
복합성 웜톤 트러블 모공 민감성 잡티 블랙헤드 
264
A000000137147
복합성 웜톤 모공 잡티 블랙헤드 트러블 주름 
265
A000000128322
복합성 웜톤 트러블 민감성 모공 탄력 블랙헤드 
266
A000000128107
복합성 웜톤 트러블 피지과다 잡티 홍조 민감성 
267
A000000135591
복합성 웜톤 모공 블랙헤드 잡티 미백 트러블 
268
A000000135374
복합성 쿨톤 모공 트러블 미백 잡티 각질 
269
A000000129772
복합성 웜톤 트러블 모공 민감성 잡티 블랙헤드 
270
A000000130397
건성 쿨톤 모공 미백 민감성 잡티 각질 
271
A000000135375
복합성 웜톤 민감성 트러블 모공 잡티 주름 
272
A000000129474
복합성 웜톤 트러블 민감성 블랙헤드 모공 잡티 
273
A000000129445
복합성 웜톤 민감성 모공 홍조 탄력 트러블 
274
A000000129444
복합성 웜톤 트러블 모공 블랙헤드 각질 민감성 
275
A000000144036
복합성 쿨톤 피지과다 잡티 트러블 모공 홍조 
276
A000000137386
복합성 웜톤 트러블 모공 잡티 민감성 블랙헤드 
277
A000000137417
복합성 쿨톤 모공 미백 잡티 주름 탄력 
278
A000000137834
복합성 웜톤 모공 트러블 잡티 각질 민감성 
279
A000000138002
건성 웜톤 모공 잡티 블랙헤드 민감성 트러블 
280
A000000124509
복합성 웜톤 잡티 트러블 주름 미백

In [16]:
df_feature

,code,feature
0,A000000147339,복합성 웜톤 트러블 모공 민감성 잡티 블랙헤드
1,A000000137180,복합성 웜톤 트러블 모공 블랙헤드 잡티 민감성
2,A000000141704,복합성 웜톤 트러블 모공 잡티 블랙헤드 민감성
3,A000000104007,복합성 웜톤 모공 트러블 잡티 민감성 블랙헤드
4,A000000146589,복합성 웜톤 트러블 모공 민감성 잡티 블랙헤드
...,...,...
369,A000000122212,건성 웜톤 잡티 미백 주름 각질 홍조
370,A000000148253,건성 웜톤 홍조 모공 잡티 민감성 각질
371,A000000122094,복합성 쿨톤 모공 블랙헤드 주름 잡티 탄력
372,A000000121274,복합성 웜톤 민감성 트러블 모공 잡티 홍조


In [17]:
df_feature
df_feature.loc[str(index)] = ('input','건성 웜톤 트러블 미백 주름')

In [18]:
df_feature.to_csv(dir+'df_feature.csv',header = True, index = True,encoding='cp949')

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

counter_vector = CountVectorizer(ngram_range=(1,3))
c_vector_features = counter_vector.fit_transform(df_feature['feature'])
c_vector_features.shape

(375, 700)

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_feature = cosine_similarity(c_vector_features,c_vector_features).argsort()[:,::-1]
print(similarity_feature)
print(similarity_feature.shape)

[[  0   4  48 ...  51 233 182]
 [ 33   1  66 ...  51 233 182]
 [343 341   2 ...  51 233 182]
 ...
 [372 366 271 ...  51 367 182]
 [373 351 260 ... 250  51 182]
 [374 180 364 ... 107 223 338]]
(375, 375)


In [21]:
def recommend_product_list(df_feature, code , top=11):
    #특정 제품코드 뽑아내기
    target_feature_index = df_feature[df_feature['code'] == code].index.values

    #타켓제품과 비슷한 코사인 유사도값
    sim_index = similarity_feature[int(target_feature_index),:top].reshape(-1)
    #본인제외
    sim_index = sim_index[sim_index!=target_feature_index]
    
    #추천 결과 새로운 df 생성, 평균평점(score)으로 정렬
    result = df_feature.iloc[sim_index]
    
    return result

In [22]:
df_result = recommend_product_list(df_feature, code='input')

In [23]:
df_result

,code,feature
374,input,건성 웜톤 트러블 미백 주름
180,A000000011084,건성 웜톤 트러블 잡티 블랙헤드 미백 민감성
364,A000000148417,건성 웜톤 트러블 모공 블랙헤드 각질 잡티
57,A000000159291,건성 웜톤 민감성 모공 잡티 트러블 미백
369,A000000122212,건성 웜톤 잡티 미백 주름 각질 홍조
214,A000000141310,건성 웜톤 트러블 모공 블랙헤드 각질 민감성
360,A000000149231,건성 웜톤 잡티 주름
212,A000000141252,건성 웜톤 탄력
187,A000000163512,건성 웜톤 각질
168,A000000156725,복합성 웜톤 주름 블랙헤드 잡티 트러블 미백


In [35]:
df_recommend = pd.merge(df_product,df_result,on='code',how='right')
df_recommend

result_dict={}
products_dict = {}

for index in df_recommend.index:
    if index == 0:
        continue
        
    product_dict = {}
    product_dict['productURL'] = str(df_recommend.loc[index]['00.상품_URL'])
    product_dict['imageURL'] = str(df_recommend.loc[index]['00.이미지_URL'])
    product_dict['brand'] = str(df_recommend.loc[index]['01.브랜드'])
    product_dict['productName'] = str(df_recommend.loc[index]['02.상품명'])
    product_dict['price'] = int(df_recommend.loc[index]['03.가격'])

    products_dict[index] = product_dict
    
result_dict['CBF'] = products_dict
print(result_dict)

{'CBF': {1: {'productURL': 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000011084&dispCatNo=100000100010008&trackingCd=Cat100000100010008_MID&curation&egcode&rccode&egrankcode', 'imageURL': 'https://image.oliveyoung.co.kr/uploads/images/goods/550/10/0000/0001/A00000001108403ko.jpg?l=ko', 'brand': '엠도씨', 'productName': '엠도씨 릴리프 토닉 위드 애프터셰이브 150ml', 'price': 20000}, 2: {'productURL': 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000148417&dispCatNo=100000100010008&trackingCd=Cat100000100010008_MID&curation&egcode&rccode&egrankcode', 'imageURL': 'https://image.oliveyoung.co.kr/uploads/images/goods/550/10/0000/0014/A00000014841703ko.jpg?l=ko', 'brand': '이너프프로젝트', 'productName': '이너프 프로젝트 에센셜 로션 150ml', 'price': 18000}, 3: {'productURL': 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000159291&dispCatNo=100000100010008&trackingCd=Cat100000100010008_MID&curation&egcode&rccode&egrankcode', 'imageURL': 'https://image

In [32]:
df_recommend

,code,00.상품_URL,00.이미지_URL,01.브랜드,02.상품명,03.가격,04.제품 주요 사양,05.모든 성분,06.총 평점,07.리뷰 개수,...,09_1.피부타입_건성,09_2.피부타입_복합성,09_3.피부타입_지성,10_1.피부고민_보습,10_2.피부고민_진정,10_3.피부고민_주름/미백,11_1.피부자극_없음,11_2.피부자극_보통,11_3.피부자극_있음,feature
0,input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,건성 웜톤 트러블 미백 주름
1,A000000011084,https://www.oliveyoung.co.kr/store/goods/getGo...,https://image.oliveyoung.co.kr/uploads/images/...,엠도씨,엠도씨 릴리프 토닉 위드 애프터셰이브 150ml,20000.0,모든 피부용,정제수사이클로펜타실록세인글리세린네오펜틸글라이콜다이헵타노에이트메틸프로판다이올하이드로제...,4.6,174.0,...,0.23,0.63,0.14,0.33,0.67,0.01,0.42,0.56,0.02,건성 웜톤 트러블 잡티 블랙헤드 미백 민감성
2,A000000148417,https://www.oliveyoung.co.kr/store/goods/getGo...,https://image.oliveyoung.co.kr/uploads/images/...,이너프프로젝트,이너프 프로젝트 에센셜 로션 150ml,18000.0,모든 피부용,정제수 부틸렌글라이콜 하이드로제네이티드폴리C614올레핀 글리세린 카프릴릭카프릭트라이...,4.6,58.0,...,0.36,0.53,0.11,0.64,0.36,0.00,0.72,0.28,0.00,건성 웜톤 트러블 모공 블랙헤드 각질 잡티
3,A000000159291,https://www.oliveyoung.co.kr/store/goods/getGo...,https://image.oliveyoung.co.kr/uploads/images/...,마몽드,마몽드 레티놀 앰플 토너 250ml,25000.0,모든피부,정제수 부틸렌글라이콜 글리세린 베타인 하이드롤라이즈드콜라겐 12헥산다이올 글리세릴폴...,4.8,64.0,...,0.58,0.42,0.00,0.70,0.13,0.17,0.78,0.20,0.02,건성 웜톤 민감성 모공 잡티 트러블 미백
4,A000000122212,https://www.oliveyoung.co.kr/store/goods/getGo...,https://image.oliveyoung.co.kr/uploads/images/...,다슈,다슈 맨즈 아쿠아 리얼 모이스트 올인원크림 153ml,18000.0,모든 피부타입,정제수 부틸렌글라이콜 식물성스쿠알란 사이클로펜타실록세인 마카다미아씨오일 나이아신아마...,4.6,111.0,...,0.34,0.56,0.10,0.67,0.30,0.04,0.60,0.40,0.00,건성 웜톤 잡티 미백 주름 각질 홍조
5,A000000141310,https://www.oliveyoung.co.kr/store/goods/getGo...,https://image.oliveyoung.co.kr/uploads/images/...,디오디너리,[디오디너리] 비타민 씨 서스펜션 30% 인 실리콘,9000.0,모든피부타입,다이메티콘 아스코빅애씨드 폴리실리콘11 피이지10다이메티콘,4.3,43.0,...,0.07,0.51,0.42,0.09,0.09,0.82,0.07,0.19,0.74,건성 웜톤 트러블 모공 블랙헤드 각질 민감성
6,A000000149231,https://www.oliveyoung.co.kr/store/goods/getGo...,https://image.oliveyoung.co.kr/uploads/images/...,오우썸,오우썸 수딩앤딥모이스처라이징 카렌듈라 토너 200ml,27000.0,모든피부용,포트마리골드꽃수포트마리골드꽃추출물포트마리골드꽃베타인글리세린서양민들레추출물옐로우체이스...,4.6,19.0,...,0.26,0.63,0.11,0.32,0.68,0.00,0.74,0.21,0.05,건성 웜톤 잡티 주름
7,A000000141252,https://www.oliveyoung.co.kr/store/goods/getGo...,https://image.oliveyoung.co.kr/uploads/images/...,진저6,진저6 라이크 화이트 블라썸 에센스 토너 150ml,32000.0,모든 피부용,정제수부틸렌글라이콜다이프로필렌글라이콜12헥산다이올글리세레스26나이아신아마이드글리세린...,4.9,15.0,...,0.53,0.33,0.14,0.67,0.13,0.20,0.73,0.20,0.07,건성 웜톤 탄력
8,A000000163512,https://www.oliveyoung.co.kr/store/goods/getGo...,https://image.oliveyoung.co.kr/uploads/images/...,식물나라,[증량]식물나라 카렌둘라 진정 토너 500ml,15800.0,모든 피부 민감성 피부,정제수 프로판다이올 다이프로필렌글라이콜 12헥산다이올 포트마리골드꽃추출물10000p...,4.9,26.0,...,0.19,0.77,0.04,0.15,0.85,0.00,0.62,0.35,0.04,건성 웜톤 각질
9,A000000156725,https://www.oliveyoung.co.kr/store/goods/getGo...,https://image.oliveyoung.co.kr/uploads/images/...,아이디얼포맨,아이디얼 포맨 빙하수 모공 애프터쉐이브 스킨 300ml,20000.0,모든 피부용,정제수 빙하수10 변성알코올 다이프로필렌글라이콜 부틸렌글라이콜 12헥산다이올 글리세...,4.7,72.0,...,0.18,0.69,0.13,0.35,0.65,0.00,0.60,0.40,0.00,복합성 웜톤 주름 블랙헤드 잡티 트러블 미백


In [25]:
# 성분 필터링
df_product['05.모든 성분']

0      약모밀추출물 정제수 12헥산다이올 글리세린 베타인 병풀추출물 체이스트트리추출물 우엉...
1      정제수 부틸렌글라이콜 글리세린 펜틸렌글라이콜 프로판다이올 아이리쉬모스추출물 사탕수수...
2      온천수 12헥산다이올 메틸프로판다이올 글리세린 나이아신아마이드 솔비톨 부틸렌글라이콜...
3      정제수 나이아신아마이드 아이소도데케인 카프릴릴메티콘 메틸프로판다이올 12헥산다이올 ...
4      토너 정제수 프로판다이올 글리세린 12헥산다이올 폴리글리세릴6카프릴레이트 폴리글리세...
                             ...                        
515    정제수다이메티콘부틸렌글라이콜글리세린트라이실록세인트레할로오스수크로오스암모늄아크릴로일다...
516                                            녹차추출물 100
517                                            녹차추출물 100
518    AHC 리얼 너리싱 세럼정제수 글리세린 프로판다이올 글리세레스26 나이아신아마이드 ...
519    정제수헥실렌글라이콜다마스크장미꽃수39800ppm글리세린페녹시에탄올알란토인소듐글루코네...
Name: 05.모든 성분, Length: 518, dtype: object

In [64]:
#최다빈출 성분
ingredient_dict = {}

for ingredient in df_product['05.모든 성분']:
    ingredient_list = ingredient.split(' ')
   
    for i in ingredient_list:

        if i in ingredient_dict:
            ingredient_dict[i] += 1
        else:
            ingredient_dict[i] = 1
ingredient_dict_sort = sorted(ingredient_dict.items(),key = lambda x: x[1],reverse = True)

print(ingredient_dict_sort[:20])

[('글리세린', 405), ('정제수', 404), ('', 367), ('부틸렌글라이콜', 358), ('12헥산다이올', 310), ('에틸헥실글리세린', 241), ('다이소듐이디티에이', 223), ('소듐하이알루로네이트', 211), ('판테놀', 179), ('프로판다이올', 177), ('향료', 176), ('트로메타민', 173), ('잔탄검', 160), ('카보머', 152), ('나이아신아마이드', 149), ('하이드로제네이티드레시틴', 137), ('펜틸렌글라이콜', 136), ('베타인', 135), ('알란토인', 129), ('리모넨', 128)]
